In [42]:
import math

import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

print(f'Using tensorflow version: {tf.__version__}')

Using tensorflow version: 1.3.0


In [2]:
mnist = input_data.read_data_sets("data/MNIST/", one_hot=True)

Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz


## Basic neural network

It only has input and a layer of activation (softmax).

In [47]:
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

y = tf.matmul(x, W) + b

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_)) # Used to avoid degenerescence


train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for _ in range(5000):
    batch_x, batch_y = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_x, y_: batch_y})
    
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy: ', sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

Accuracy:  0.9247


### Some helper functions

In [51]:
def weight_variable(shape):
    """
    Produce a weight variable adapted for ReLU neurons.
    It will have values following a normal distribution, centered to
    0.0 and having all absolute values bigger than 2*stddev dropped
    and re-picked.
    """
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    """
    Produce slightly positive bias to avoid 'dead neurons'.
    """
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def leaky_relu(x, eps=0.01):
    """
    If x > 0 -> x
    Else x * eps.
    """
    return tf.maximum(x, x * eps)

## Deeper neural network

Now with hidden layers!

In [53]:
x = tf.placeholder(tf.float32, [None, 784])
xx = tf.reshape(x, [-1, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

lr = tf.placeholder(tf.float32) # Variable learning rate
pkeep = tf.placeholder(tf.float32) # Proba to keep neuron during dropout

# Hidden layer 01
W0 = weight_variable([784, 200])
b0 = bias_variable([200])
Y0 = tf.nn.relu(tf.matmul(xx, W0) + b0)
Y0_ = tf.nn.dropout(Y0, pkeep)

# Hidden layer 02
W1 = weight_variable([200, 50])
b1 = bias_variable([50])
Y1 = tf.nn.relu(tf.matmul(Y0_, W1) + b1)
Y1_ = tf.nn.dropout(Y1, pkeep)

W2 = weight_variable([50, 10])
b2 = bias_variable([10])
y = tf.matmul(Y1_, W2) + b2

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_)) # Used to avoid degenerescence


train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for i in range(5000):
    max_learning_rate = 0.003
    min_learning_rate = 0.0001
    decay_speed = 2000.0
    learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-i/decay_speed)
    
    batch_x, batch_y = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_x, y_: batch_y, lr: learning_rate, pkeep: .75})
    
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy: ', sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels, pkeep: 1}))

Accuracy:  0.9807
